In [1]:
import pandas as pd
import numpy as np
import censusdata

In [3]:
#censusdata.search('acs5', 2015, 'concept', 'race')

In [5]:
censusdata.printtable(censusdata.censustable('acs5', 2015, 'B02001'))

Variable     | Table                          | Label                                                    | Type 
-------------------------------------------------------------------------------------------------------------------
B02001_001E  | B02001.  Race                  | Total:                                                   | int  
B02001_002E  | B02001.  Race                  | White alone                                              | int  
B02001_003E  | B02001.  Race                  | Black or African American alone                          | int  
B02001_004E  | B02001.  Race                  | American Indian and Alaska Native alone                  | int  
B02001_005E  | B02001.  Race                  | Asian alone                                              | int  
B02001_006E  | B02001.  Race                  | Native Hawaiian and Other Pacific Islander alone         | int  
B02001_007E  | B02001.  Race                  | Some other race alone                        

In [42]:
df = censusdata.download('acs5', 2011, censusdata.censusgeo([('state', '25'), ('county', '025'), ('tract', '*')]),
                             ['B02001_008E'])
df = df.reset_index(drop=False)
df.rename(columns = {'index' : 'TRACT', 'B02001_008E' : 'Two or More Races'}, inplace = True)

for x in range(len(df)):
    df.loc[x, 'TRACT'] = df['TRACT'].unique()[x].geo[2][1]
    
df = df.astype({"TRACT": int})

df

,TRACT,Two or More Races
0,20303,48
1,40401,87
2,40801,70
3,50101,1211
4,50901,1275
...,...,...
199,101101,13
200,130300,147
201,140102,170
202,140300,294


In [43]:
template = pd.read_csv('Precinct_Make_Up_Of_Census_Tracts.csv')
template = template.drop('Unnamed: 0', axis=1)

prct_age = pd.DataFrame(0, index=np.arange(len(template.columns[1:])),
                        columns=df.columns[1:])

prct_age.insert(0, 'WARD_PRECI', template.columns[1:])

for i in range(len(prct_age)):
    prct = prct_age['WARD_PRECI'].loc[i]
    temp = template[['TRACT', prct]]
    for n in range(len(temp)):
        if temp[prct].loc[n] != 0:
            dec = temp.loc[n, prct]
            filt = df['TRACT'] == temp.loc[n, 'TRACT']
            for c in df.columns[1:]:
                prct_age[c].iloc[i] += round(df[filt][c] * dec)

/Users/yagevlevi/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/Users/yagevlevi/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/Users/yagevlevi/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

In [24]:
df = pd.read_csv('City Council/2009_November Municipal (General) City Council At Large Results.csv')
df.insert(22, 'TWO OR MORE RACES', np.zeros(len(df)))
for i in range(len(prct_age)):
    try:
        wp = int(prct_age.loc[i, 'WARD_PRECI'])
    except ValueError:
        wp = 502
    filt = df['Ward/Precinct'] == wp
    index = df.index
    ind = index[filt].tolist()[0]
    df.loc[ind, 'TWO OR MORE RACES'] += round(prct_age.loc[i, 'Two or More Races']/df.loc[ind, 'TOTAL POP.'], 3)
    

df = df.drop('Unnamed: 0', axis=1)
display(df)
df.to_csv('City Council/2009_November Municipal (General) City Council At Large Results.csv')

,Ward/Precinct,STEPHEN J MURPHY,DOUG BENNETT,FELIX G ARROYO,TITO JACKSON,TOMAS GONZALEZ,JOHN R CONNOLLY,ANDREW P KENNEALLY,AYANNA S PRESSLEY,ALL OTHERS,...,Registered Voters,CVAP,TOTAL POP.,WHITE,AFRICAN_AMERICAN,NATIVE_AMERICAN,ASIAN,HISPANIC,TWO OR MORE RACES,OTHER
0,101,175,62,116,63,41,188,186,73,5,...,1113,534.0,960.039645,0.567,0.033,0.000,0.100,0.403,0.154,0.146
1,102,147,52,71,25,43,129,115,41,4,...,816,814.0,2041.544654,0.667,0.023,0.001,0.057,0.538,0.227,0.026
2,103,242,88,217,93,138,217,195,127,2,...,1720,1422.0,3134.463144,0.650,0.062,0.003,0.051,0.465,0.186,0.048
3,104,78,44,38,18,11,72,73,25,0,...,563,537.0,1721.867952,0.567,0.073,0.003,0.032,0.548,0.263,0.062
4,105,116,54,82,23,34,110,157,37,4,...,960,980.0,3189.980535,0.645,0.032,0.008,0.001,0.640,0.216,0.099
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249,2209,144,46,106,70,30,153,78,111,2,...,1392,1190.0,1524.975326,0.875,0.019,0.000,0.098,0.039,0.008,0.001
250,2210,276,56,189,72,44,235,125,171,6,...,1664,1655.0,2258.588637,0.702,0.057,0.001,0.163,0.133,0.006,0.072
251,2211,190,33,118,45,25,163,76,89,2,...,1079,1730.0,2101.022294,0.793,0.033,0.003,0.123,0.117,0.008,0.041
252,2212,211,46,127,47,37,186,103,91,2,...,1100,1604.0,2351.383922,0.617,0.067,0.000,0.184,0.199,0.007,0.125


In [34]:
df = pd.read_csv('City Council/2019_November Municipal (General) City Council At Large Results.csv')
rounded = round(df['TOTAL POP.'])
df['TOTAL POP.'] = rounded
df = df.drop('Unnamed: 0', axis=1)
#df = df.drop('Unnamed: 0.1', axis=1)
#df = df.drop('Unnamed: 0.1.1', axis=1)
display(df)
df.to_csv('City Council/2019_November Municipal (General) City Council At Large Results.csv')

,Ward/Precinct,MICHAEL F FLAHERTY,ALEJANDRA NICOLE STGUILLEN,MICHELLE WU,JULIA MEJIA,ERIN J MURPHY,ALTHEA GARRISON,DAVID HALBERT,ANNISSA ESSAIBI GEORGE,ALL OTHERS,...,Registered Voters,CVAP,TOTAL POP.,WHITE,AFRICAN_AMERICAN,NATIVE_AMERICAN,ASIAN,TWO OR MORE RACES,HISPANIC,OTHER
0,101,119,92,180,93,51,54,136,163,4,...,1485,889.0,1194.0,0.766,0.019,0.005,0.039,0.132,0.304,0.024
1,102,72,53,86,55,29,19,42,73,0,...,1108,1286.0,2632.0,0.693,0.038,0.001,0.039,0.170,0.536,0.058
2,103,174,145,207,167,64,59,104,175,4,...,2665,1908.0,3500.0,0.735,0.039,0.005,0.037,0.128,0.500,0.055
3,104,52,57,79,54,22,19,36,66,4,...,1122,1015.0,2318.0,0.661,0.069,0.001,0.036,0.214,0.589,0.019
4,105,65,57,78,50,35,18,42,70,1,...,1376,1169.0,4017.0,0.559,0.014,0.000,0.012,0.378,0.700,0.036
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249,2209,111,86,161,62,56,45,45,177,2,...,1362,1185.0,1521.0,0.766,0.017,0.000,0.128,0.047,0.101,0.041
250,2210,146,125,245,98,74,82,50,207,3,...,1583,1904.0,2604.0,0.726,0.056,0.000,0.113,0.038,0.163,0.063
251,2211,152,69,157,64,74,69,28,128,2,...,1068,2046.0,2549.0,0.754,0.058,0.000,0.087,0.043,0.099,0.058
252,2212,155,86,155,69,85,71,38,135,0,...,1257,2052.0,2985.0,0.678,0.063,0.000,0.132,0.037,0.229,0.090


In [44]:
df = pd.read_csv('Mayoral Election/2009_November Municipal (General) Mayoral Results.csv')
#df['TWO OR MORE RACES'] = np.zeros(len(df))
df.insert(17, 'TWO OR MORE RACES', np.zeros(len(df)))
for i in range(len(prct_age)):
    try:
        wp = int(prct_age.loc[i, 'WARD_PRECI'])
    except ValueError:
        wp = 502
    filt = df['Ward Precinct '] == wp
    index = df.index
    ind = index[filt].tolist()[0]
    df.loc[ind, 'TWO OR MORE RACES'] += round(prct_age.loc[i, 'Two or More Races']/df.loc[ind, 'TOTAL POP.'], 3)
    

df = df.drop('Unnamed: 0', axis=1)
df = df.drop('Unnamed: 0.1', axis=1)
display(df)
df.to_csv('Mayoral Election/2009_November Municipal (General) Mayoral Results.csv')

,Ward Precinct,THOMAS M MENINO,MICHAEL F FLAHERTY,WILLIAM FEEGBEH (WRITE-IN),ALL OTHERS,VOTES CAST,BLANKS,BALLOTS CAST,Registered Voters,CVAP,TOTAL POP.,WHITE,AFRICAN_AMERICAN,NATIVE_AMERICAN,ASIAN,TWO OR MORE RACES,HISPANIC,OTHER
0,101,200,193,0,0,393,4,397,1113,534.0,960.0,0.567,0.033,0.000,0.100,0.154,0.403,0.146
1,102,166,105,0,2,273,0,273,816,814.0,2042.0,0.667,0.023,0.001,0.057,0.227,0.538,0.026
2,103,365,229,0,0,594,6,600,1720,1422.0,3134.0,0.650,0.062,0.003,0.051,0.186,0.465,0.048
3,104,115,66,0,0,181,2,183,563,537.0,1722.0,0.567,0.073,0.003,0.032,0.263,0.548,0.062
4,105,164,121,0,0,285,1,286,960,980.0,3190.0,0.645,0.032,0.008,0.001,0.216,0.640,0.099
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249,2209,147,155,0,0,302,2,304,1392,1190.0,1525.0,0.875,0.019,0.000,0.098,0.008,0.039,0.001
250,2210,244,220,0,3,467,3,470,1664,1655.0,2259.0,0.702,0.057,0.001,0.163,0.006,0.133,0.072
251,2211,168,142,0,1,311,5,316,1079,1730.0,2101.0,0.793,0.033,0.003,0.123,0.008,0.117,0.041
252,2212,192,171,0,2,365,7,372,1100,1604.0,2351.0,0.617,0.067,0.000,0.184,0.007,0.199,0.125


In [66]:
df = censusdata.download('acs5', 2019, censusdata.censusgeo([('state', '25'), ('county', '025'), ('tract', '*')]),
                             ['B02001_001E', 'B02001_002E'])
df = df.reset_index(drop=False)
df.rename(columns = {'index' : 'TRACT', 'B02001_001E' : 'Total', 'B02001_002E' : 'White'}, inplace = True)

for x in range(len(df)):
    df.loc[x, 'TRACT'] = df['TRACT'].unique()[x].geo[2][1]
    
df = df.astype({"TRACT": int})

bipoc = df['Total'] - df['White']

df['BIPOC'] = bipoc

df = df[['TRACT', 'BIPOC']]

df

,TRACT,BIPOC
0,802,2804
1,10600,992
2,40100,73
3,40300,723
4,50400,519
...,...,...
199,82100,5278
200,60200,32
201,90400,4099
202,120500,1294


In [67]:
template = pd.read_csv('Precinct_Make_Up_Of_Census_Tracts.csv')
template = template.drop('Unnamed: 0', axis=1)

prct_age = pd.DataFrame(0, index=np.arange(len(template.columns[1:])),
                        columns=df.columns[1:])

prct_age.insert(0, 'WARD_PRECI', template.columns[1:])

for i in range(len(prct_age)):
    prct = prct_age['WARD_PRECI'].loc[i]
    temp = template[['TRACT', prct]]
    for n in range(len(temp)):
        if temp[prct].loc[n] != 0:
            dec = temp.loc[n, prct]
            filt = df['TRACT'] == temp.loc[n, 'TRACT']
            for c in df.columns[1:]:
                prct_age[c].iloc[i] += round(df[filt][c] * dec)

/Users/yagevlevi/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/Users/yagevlevi/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/Users/yagevlevi/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

In [58]:
df = pd.read_csv('City Council/2019_November Municipal (General) City Council At Large Results.csv')
df.insert(22, 'BIPOC', np.zeros(len(df)))
for i in range(len(prct_age)):
    try:
        wp = int(prct_age.loc[i, 'WARD_PRECI'])
    except ValueError:
        wp = 502
    filt = df['Ward/Precinct'] == wp
    index = df.index
    ind = index[filt].tolist()[0]
    df.loc[ind, 'BIPOC'] += round(prct_age.loc[i, 'BIPOC']/df.loc[ind, 'TOTAL POP.'], 3)
    

df = df.drop('Unnamed: 0', axis=1)
display(df)
df.to_csv('City Council/2019_November Municipal (General) City Council At Large Results.csv')

,Ward/Precinct,MICHAEL F FLAHERTY,ALEJANDRA NICOLE STGUILLEN,MICHELLE WU,JULIA MEJIA,ERIN J MURPHY,ALTHEA GARRISON,DAVID HALBERT,ANNISSA ESSAIBI GEORGE,ALL OTHERS,...,CVAP,TOTAL POP.,WHITE,AFRICAN_AMERICAN,NATIVE_AMERICAN,ASIAN,TWO OR MORE RACES,BIPOC,HISPANIC,OTHER
0,101,119,92,180,93,51,54,136,163,4,...,889.0,1194.0,0.766,0.019,0.005,0.039,0.132,0.234,0.304,0.024
1,102,72,53,86,55,29,19,42,73,0,...,1286.0,2632.0,0.693,0.038,0.001,0.039,0.170,0.306,0.536,0.058
2,103,174,145,207,167,64,59,104,175,4,...,1908.0,3500.0,0.735,0.039,0.005,0.037,0.128,0.265,0.500,0.055
3,104,52,57,79,54,22,19,36,66,4,...,1015.0,2318.0,0.661,0.069,0.001,0.036,0.214,0.340,0.589,0.019
4,105,65,57,78,50,35,18,42,70,1,...,1169.0,4017.0,0.559,0.014,0.000,0.012,0.378,0.441,0.700,0.036
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249,2209,111,86,161,62,56,45,45,177,2,...,1185.0,1521.0,0.766,0.017,0.000,0.128,0.047,0.233,0.101,0.041
250,2210,146,125,245,98,74,82,50,207,3,...,1904.0,2604.0,0.726,0.056,0.000,0.113,0.038,0.274,0.163,0.063
251,2211,152,69,157,64,74,69,28,128,2,...,2046.0,2549.0,0.754,0.058,0.000,0.087,0.043,0.246,0.099,0.058
252,2212,155,86,155,69,85,71,38,135,0,...,2052.0,2985.0,0.678,0.063,0.000,0.132,0.037,0.322,0.229,0.090


In [69]:
df = pd.read_csv('Mayoral Election/2017_November Municipal (General) Mayoral Results.csv')
#df['TWO OR MORE RACES'] = np.zeros(len(df))
df.insert(17, 'BIPOC', np.zeros(len(df)))
for i in range(len(prct_age)):
    try:
        wp = int(prct_age.loc[i, 'WARD_PRECI'])
    except ValueError:
        wp = 502
    filt = df['Ward_Precinct'] == wp
    index = df.index
    ind = index[filt].tolist()[0]
    df.loc[ind, 'BIPOC'] += round(prct_age.loc[i, 'BIPOC']/df.loc[ind, 'TOTAL POP.'], 3)
    

df = df.drop('Unnamed: 0', axis=1)
#df = df.drop('Unnamed: 0.1', axis=1)
display(df)
df.to_csv('Mayoral Election/2017_November Municipal (General) Mayoral Results.csv')

,Ward_Precinct,TITO JACKSON,MARTIN J WALSH,DONALD OSGOOD SR (WRITE-IN),ALL OTHERS,VOTES CAST,BLANKS,BALLOTS CAST,Registered Voters,CVAP,TOTAL POP.,WHITE,AFRICAN_AMERICAN,NATIVE_AMERICAN,ASIAN,TWO OR MORE RACES,BIPOC,HISPANIC,OTHER
0,101.0,162.0,315.0,0.0,4.0,481.0,14.0,495.0,1397.0,889.0,1194.0,0.766,0.019,0.005,0.039,0.131,0.234,0.304,0.024
1,102.0,98.0,189.0,0.0,3.0,290.0,8.0,298.0,1035.0,1286.0,2632.0,0.693,0.038,0.001,0.039,0.170,0.306,0.536,0.058
2,103.0,233.0,453.0,0.0,6.0,692.0,19.0,711.0,2379.0,1908.0,3500.0,0.735,0.039,0.005,0.037,0.128,0.265,0.500,0.055
3,104.0,59.0,162.0,0.0,2.0,223.0,7.0,230.0,798.0,1015.0,2318.0,0.661,0.069,0.001,0.036,0.214,0.340,0.589,0.019
4,105.0,87.0,254.0,0.0,3.0,344.0,11.0,355.0,1255.0,1169.0,4017.0,0.559,0.014,0.000,0.012,0.378,0.441,0.700,0.036
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249,2209.0,99.0,177.0,0.0,3.0,279.0,5.0,284.0,1328.0,1185.0,1521.0,0.766,0.017,0.000,0.128,0.047,0.233,0.101,0.041
250,2210.0,149.0,237.0,0.0,2.0,388.0,3.0,391.0,1455.0,1904.0,2604.0,0.726,0.056,0.000,0.113,0.038,0.274,0.163,0.063
251,2211.0,90.0,168.0,0.0,2.0,260.0,5.0,265.0,1041.0,2046.0,2549.0,0.754,0.058,0.000,0.087,0.043,0.246,0.099,0.058
252,2212.0,84.0,196.0,0.0,2.0,282.0,2.0,284.0,1200.0,2052.0,2985.0,0.678,0.063,0.000,0.132,0.037,0.322,0.229,0.090
